In [5]:
#初始化
import os
from openai import AzureOpenAI

#pypandoc.download_pandoc()

client = AzureOpenAI(
  api_key = "0ac58578134f428885682a58a596a569",  
  api_version = "2023-05-15",
  azure_endpoint = "https://haxudalle.openai.azure.com/"
)

# 设置openai API默认参数, name = gpt-35-turbo-1106
def get_completion(prompt): 
    messages = [
        {"role": "system", "content": """
         Translate this markdown content to Simplified Chinese. Follow the below rules:
         1. Must keep original markdown format and position. 
         2. Do not change any markdown or html tags.
         3. The expected documentation or material shall keep the integrality of contents.
         4. The translated content shall be grammatically correct.
         5. The translated content shall be fluent and easy to understand.
         6. Must follow the below glossary which between two ###.
         ###
         prompt:提示
         prompts:提示词
         prompting:提示词
         prompt engeieering:提示工程
         zero shot:零示例
         one shot:单示例
         few shot:少示例
         CoT:思维链
         ###
         """},
        {"role": "user", "content": prompt}
        ]
    response = client.chat.completions.create(
      model="gpt-35-turbo",
      #model="gpt-4-1106-preview",
      messages=messages
      )
    return response.choices[0].message.content


In [3]:
import os
#汉化文件夹中所有md文件
def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def save_to_markdown(file_path, content):
    base_name = os.path.basename(file_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_chs{ext}"
    new_path = os.path.join(os.path.dirname(file_path), new_name)
    with open(new_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_and_save_all_markdown_files(directory_path):
    processed_files_count = 0
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.md' or '.MD'):
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, directory_path)
                print("Processing file:", relative_path)
                content = read_markdown_file(file_path)
                processed_content = get_completion(content)
                save_to_markdown(file_path, processed_content)
                processed_files_count += 1
    return processed_files_count

directory_path = 'labs'
processed_files_count = process_and_save_all_markdown_files(directory_path)

In [6]:
##翻译ipynb文件中的所有markdown单元格
import os
import nbformat

def process_and_save_all_ipynb_files(directory_path):
    processed_files_count = 0
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.ipynb'):
                file_path = os.path.join(root, file)
                process_and_save_ipynb(file_path)
                processed_files_count += 1
    return processed_files_count

def process_and_save_ipynb(file_path):
    # 读取.ipynb文件
    with open(file_path, 'r', encoding='utf-8') as f:
        nb = nbformat.read(f, as_version=4)

    # 遍历所有单元格
    for i, cell in enumerate(nb.cells):
        # 如果单元格是Markdown类型
        if cell.cell_type == 'markdown':
            # 使用get_completion函数处理单元格内容
            cell.source = get_completion(cell.source)

    # 生成新的文件名
    base_name = os.path.basename(file_path)
    name, ext = os.path.splitext(base_name)
    new_name = f"{name}_chs{ext}"
    new_path = os.path.join(os.path.dirname(file_path), new_name)

    # 保存.ipynb文件
    with open(new_path, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)

# 使用函数
directory_path = './test/'
processed_files_count = process_and_save_all_ipynb_files(directory_path)

In [ ]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, chunk_size=1000, chunk_overlap=0
)

def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def save_to_markdown(file_path, content):
    with open(file_path, 'a', encoding='utf-8') as f:
        f.write(content)

file_path = 'moaw\workshops\github-copilot\workshop.md'
markdown_text = read_markdown_file(file_path)

md_docs = md_splitter.create_documents([markdown_text])

for doc in md_docs:
    doc.page_content = get_completion(doc.page_content)

    # 保存处理后的内容到新的 Markdown 文件
    new_file_path = file_path.replace('.md', '_chs.md')
    save_to_markdown(new_file_path, doc.page_content)
